# Load API key:

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Libraries:

In [21]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
import json

In [5]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

In [6]:
import openai

# Embeddings:

In [7]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Vector database:

In [35]:
persist_directory = "db"
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)

In [22]:
with open('D:/work/webautomation/write_test.json', 'r') as f:
    data = json.load(f)
topic_list=[]
for i in data:
  topic_list.append(i["title"])
print(len(topic_list))

persist_directory='db'
for i in data:
  # Chunking
  doc=i["body"]
  texts=text_splitter.split_text(doc)
  print(texts)
  # print(doc[:10])
  # upserting
  # vectordb=Chroma.from_texts(texts, embedding=embeddings,persist_directory=persist_directory)
  for t in texts:
    vectordb=Chroma.from_texts([t], embedding=embeddings,persist_directory=persist_directory,metadatas={"source":i["url"],"Topic":i["title"]})
    vectordb.persist()
    vectordb = None

19
["Write tests for websites or web appsOverviewNightwatch provides simple and comprehensive APIs for interacting with web elements and performing various actions and assertions. In Nightwatch v3, brand new element APIs are introduced to make writing tests even simpler and more concise. The Nightwatch inspector makes it easier to author tests as it provides selector recommendations that are durable in the longer run. If you know very little or no coding, you can also use our Chrome recorder extension to record tests and run them using Nightwatch. Finding elementsBefore you can interact with elements or perform assertions, you will have to find the elements from the DOM tree using selectors. Nightwatch supports a variety of selectors to make finding elements a breeze. You can do so using .find() & .findAll() related commands:CSS selectors :browser.element.find('css selector');browser.element.findAll('css selector');xPath selector :browser.element.find(by.xpath(('xpath string'));browser

["Selectors for websites & web appsOverviewThe 1st step towards writing any web interaction is to find an element. Selectors are needed to find the element from the DOM tree. With Nightwatch v3, the selectors have been upgraded to make it even more simpler to find elements. V3 also introduces chaining of .find() commands to be able to locate nested elements with ease.CSS SelectorsYou can easily find elements within the DOM tree using CSS selectors. There are multiple types of simple & complex CSS selectors that Nightwatch supports. Some common examples includeid basedclass basedelement class basednested// Find an element which contains id = element-idbrowser.element.find('#element-id');// Find all elements with CSS class activebrowser.element.findAll('.active');// Find all <p> elements with class activebrowser.element.find('p.active');// Find element with class active nested within an element with id listbrowser.element.find('#id > .active');// Find an element which contains id = eleme

["BDD test syntaxOverviewStarting with Nightwatch version 1.3, you can use the popular BDD interfaces for writing tests. You don't need additional configuration to use BDD interfaces. These are now supported out of the box.You can also run tests written in BDD describe and Exports interfaces together. Prior to this version, you had to use the Mocha test runner for enabling this functionality, which is now possible without additional plugins or libraries. The BDD interface in Nightwatch provides the following functions:describe() / context()test() / it() / specify()before()after()beforeEach()afterEach()Nightwatch doesn't support nested describe`/`context declarations currently. You can only use describe to define the name for the test suite. Exampledescribe('Ecosia', function() {    // test() and specify() is also available    it('demo test', function(browser) {    browser      .url('https://www.ecosia.org/')      .setValue('input[type=search]', 'nightwatch')      .click('button[type=su

["Using ES6 async/awaitOverviewStarting with Nightwatch version 1.1, you can write tests as an ES6 async function.The async function enables the API commands to return a promise and makes it possible to use the await operator to retrieve the result, instead of the callback, as it is by default.Use of async function greatly improves the readability and ease of writing of tests. Starting Nightwatch version 1.7, chaining the API commands when using an async function is also supported.Exampletests/exampleTest.jsmodule.exports = {  'demo test async': async function (browser) {    // get the available window handles    const result = await browser.windowHandles();    console.log('result', result);        // switch to the second window    // await is not necessary here since we're not interested in the result    browser.switchWindow(result.value[1]);  }};Using Callbacks with asyncCallbacks can still be used as before and if the callback returns a Promise, the result of the promise is the resu

["Write complex user actionsOverviewNightwatch 2 brings support for working with the newer Actions API from Selenium WebDriver for performing complex user gestures.The Actions API provides granular control over exactly what designated input devices can do. Selenium provides an interface for 3 kinds of input sources: a key input for keyboard devicesa pointer input for a mouse, pen or touch deviceswheel inputs for scroll wheel devices (introduced in Selenium 4.2)More information is available on the W3C Webdriver spec page.ExampleThe new API is available and ready to use in Nightwatch via the existing .perform() command. The previous functionality of the perform() command is still there and working in the same way as before.tests/sampleTest.jsdescribe('user actions api', function() {    it('demo test', function() {    browser      .navigateTo('https://nightwatchjs.org')      .perform(function() {        const actions = this.actions({async: true});                return actions         .ke

['Using test globalsAnother useful concept that Nightwatch provides is test globals. In its most simple form, it is a dictionary of name-value pairs which is defined in your configuration file.Globals can be defined either as a "globals" property or as an external file which is specified as the "globals_path" property.Here\'s an example definition using the "globals" property in nightwatch.json:nightwatch.conf.js{  "src_folders": [],  "globals": {    "myGlobalVar" : "some value",    "otherGlobal" : "some other value"  },  "test_settings": {    "default": {      "launch_url": "https://nightwatchjs.org",    }  }}Like the launch_url property, the globals object is made available directly on the Nightwatch api which is passed to the tests.tests/sampleTest.jsmodule.exports = {  \'Demo test\' : function (browser) {    console.log(browser.globals.myGlobalVar); // myGlobalVar == "some value"  }};Pre-defined GlobalsThe following global properties can be used to control the behaviour of the test

["Visual Regression Testing (VRT) using NightwatchOverviewVisual Regression Testing (VRT) is a type of software testing technique that compares screenshots or images of a user interface before and after changes have been made to the application. The goal of VRT is to detect unintended visual changes that may have been introduced during development, such as changes to layout, colour, typography, or other visual elements.Visual Regression Testing typically involves the following steps:Capture screenshots of the application or webpage before and after changes have been made.Compare the screenshots pixel-by-pixel to identify any differences.Review and analyse the differences to determine whether they are intentional or unintentional.Approve changes if they are intentionalHow does it work?Nightwatch uses JIMP, a Javascript image processing library with no native dependencies, to do the image comparison.Nightwatch logic:Waits for the elements to be presentTakes a screenshotCompares screensho

['Writing Unit Tests with NightwatchOverviewUnit testing in Nightwatch has been available since version 0.9 and unit tests written in Nightwatch are also fully compatible with Mocha\'s Exports interface, so you can use either test runners.Unit Tests ModeNightwatch automatically attempts to connect to the WebDriver server and create a session. When running unit tests this needs to be disabled and the runner needs to be made aware that it is operating in unit testing mode.This can be done in two ways:1. Setting unit_tests_mode=trueThis is a global option. Set the unit_tests_mode option to true in the nightwatch.json:{  "src_folders" : ["tests"],    "unit_tests_mode": true}2. Adding @unitTest property per testYou can set the @unitTest property to true if you\'d like to have individual test suites as unit tests.tests/sampleTest.jsconst assert = require(\'assert\');module.exports = {  \'@unitTest\': true,    \'demo UnitTest\' : function (done) {    assert.equal(\'TEST\', \'TEST\');    setTi

['Nightwatch InspectorOverviewThe Nightwatch inspector is designed to simplify authoring tests. Major advantages of using this are:Save time identifying selectorsBuild more robust tests Reduced back and forth between test file & browser as you can test commands in the browser itselfStart Nightwatch InspectorFollow these to start the Nightwatch Inspector:Step 1 - Add debug lineAdd browser.debug() in the test at a line where you want to start the Nightwatch inspector.Step 2 - Run test with debug flagRun the specific Nightwatch test with --debug test:npx nightwatch /test/inspector_demo.js --env chrome --debugStep 3 - Open Nightwatch selectorRunning the test would have opened up a browser and it would be paused at the line where .debug() was present. Now open dev-tools, click on >> and select Nightwatch Inspector.Explore modeTo get selector recommendations, you have to enable the explore mode. Once the explore mode is enabled and you point and click in the browser window, you will get sele

# Examples:

In [29]:
outs_dir = os.getcwd() + "\\nightwatch_outputs"
ques_dir = os.getcwd() + "\\nightwatch_queries"
examples = []
queries = []
outputs = []

for file in os.listdir(ques_dir):
    with open(os.path.join(ques_dir, file), "r") as f:
        queries.append(f.read()) 
for file in os.listdir(outs_dir):
    with open(os.path.join(outs_dir, file), "r") as f:
        outputs.append(f.read())       

for que, out in zip(queries, outputs):
    dictionary = {'query': que,'output': out}
    examples.append(dictionary)

In [30]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=3 # number of examples to produce
)

# Custom prompt:

In [31]:
example_prompt = PromptTemplate(input_variables=["query", "output"], template="Query: {query}\nOutput: {output}")

In [32]:
custom_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["context", "steps"],
    prefix="Consider the following steps:\n{steps}\nThese represent the steps in automation testing of a website. Your main task is to convert them into nightwatch javascript format. Check the examples below as reference.",
    suffix="\nNow, to begin the task, take a look at the set of commands in nighwatch\n{context}.\nAfter you determine the correct commands to use for given query, return the answer in the same format as examples. Don't write any comments in the final output"
)

# Chain:

In [33]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name='text-davinci-003')
chain_json = load_qa_chain(llm, chain_type="stuff", prompt=custom_prompt)

In [36]:
query = input("Enter the steps:")
docs = vectordb.similarity_search(query=query, distance_metric="dot", k = 4)
out = chain_json({"input_documents": docs, "steps": query},  return_only_outputs=True)

Enter the steps:1. go to youtube.com


ValueError: unexpected '{' in field name

In [ ]:
out